In [6]:
import numpy as np
import matplotlib.pyplot as plt
import databehandling.matlib as matlib
import matplotlib
from lib import laminatelib, matlib
import matplotlib.pyplot as plt

In [1]:
# Functions

def S2D(m):
    return np.array([[        1/m['E1'], -m['v12']/m['E1'],          0],
                     [-m['v12']/m['E1'],         1/m['E2'],          0],
                     [                0,                 0, 1/m['G12']]], float)

def Q2D(m):
    S=S2D(m)
    return np.linalg.inv(S)


def T2Ds(a):
    c,s = np.cos(np.radians(a)), np.sin(np.radians(a))
    return np.array([[ c*c ,  s*s ,   2*c*s],
                     [ s*s ,  c*c ,  -2*c*s],
                     [-c*s,   c*s , c*c-s*s]], float)

# Strain transform, global to local
def T2De(a):
    c,s = np.cos(np.radians(a)), np.sin(np.radians(a))
    return np.array([[   c*c,   s*s,     c*s ],
                     [   s*s,   c*c,    -c*s ],
                     [-2*c*s, 2*c*s, c*c-s*s ]], float)

def Q2Dtransform(Q,a):
    return np.dot(np.linalg.inv( T2Ds(a) ) , np.dot(Q,T2De(a)) )

def laminateThickness(layup):
    return sum([layer['thi'] for layer in layup])

def fE_max_stress(s, m):
    return max(s[0]/m["XT"], -s[0]/m["XC"])
    


def computeA(layup):
    A=np.zeros((3,3),float)
    hbot = -laminateThickness(layup)/2        # bottom of first layer
    for layer in layup:
        m = layer['mat']
        Q = Q2D(m)
        a = layer['ori']
        Qt = Q2Dtransform(Q,a)
        htop = hbot + layer['thi']   # top of current layer
        A = A + Qt*(htop-hbot)
        hbot=htop                    # for the next layer
    return A

def fE_hashin(s,m):
    s1,s2,s3,s23,s13,s12=s[0],s[1],s[2],s[3],s[4],s[5]
    XT,YT,ZT,XC,YC,ZC,S12,S13,S23 = m['XT'], m['YT'], m['ZT'], m['XC'], m['YC'], m['ZC'], m['S12'], m['S13'], m['S23']
    if s1>0:
        R = ( 1/( (s1/XT)**2 + (1/S12**2)*(s12**2 + s13**2) ) )**0.5
        fE_FF=1/R
    if s1<=0:
        fE_FF=-s1/XC
    if (s2+s3)>=0:
        temp=( (1/YT**2)*(s2+s3)**2+(1/S23**2)*(s23**2-s2*s3)+(1/S12**2)*(s12**2+s13**2) )
        if temp==0:
            fE_IFF = 0
        else:
            R = (1/temp)**0.5
            fE_IFF = 1/R
    if (s2+s3)<0:
        b = (1/YC)*((YC/(2*S23))**2-1)*(s2+s3)
        a = (1/(4*S23**2))*(s2+s3)**2+(1/S23**2)*(s23**2-s2*s3)+(1/S12**2)*(s12**2+s13**2)
        if a==0:
            fE_IFF = 0.0
        else:
            c=-1
            R=(-b+(b**2-4*a*c)**0.5)/(2*a)
            fE_IFF = 1/R
    return fE_FF, fE_IFF

def fE_hashin_2D(s, m):
    sV = s
    YT = m["YT"]
    S12 = m["S12"]
    YC = m["YC"]
    S23 = m["S23"]
    if(sV[1] >= 0):
        temp = ((1/YT**2)*sV[1]**2)+((1/S12**2)*sV[2]**2)
        if temp == 0:
            fe = 0
        else:
            R = (1/temp)**0.5
            fe = 1/R
    if(sV[1] < 0):
        #solving with abc-formula
        a = (sV[1]**2/(4*S23**2))+(sV[2]**2/S12**2)
        b = (1/YC)*(((YC**2/(4*S23**2))-1)*sV[1])
        c = -1
        if a == 0: #avoid dividing by zero
            fe = 0
        else:
            R = (-b+(b**2-4*a*c)**0.5)/(2*a)
            fe = 1/R
    return 0, fe


def solve(layup, R, P, Fx, T):
    t = laminateThickness(layup)
    Nx=P*R/2 + Fx/(2*np.pi*R)
    Ny=P*R
    Ip = 2*np.pi*R**3*t
    Nxy = T*R*t/Ip

    loads=[Nx,Ny,Nxy]
    A = computeA(layup)
    strains_global=np.linalg.solve(A,loads)
    
    fE_ff = 0
    fE_iff = 0
    fE_iff_2D = 0
    stresses = []
    local_strains = []
    Qs = []
    global_strains = []
    global_stresses = []
    results = {}
    
    for layer in layup:
        angle = layer["ori"]
        strain_local = np.matmul(T2De(angle), strains_global)
        mat = layer["mat"]
        Q = Q2D(mat)
        Q_global = Q2Dtransform(Q, angle)
        
        s = np.matmul(Q, strain_local)
        s_whole = [s[0], s[1], 0, 0, 0, s[2]]
        s_global = np.matmul(Q_global, strains_global)
        
        fE_ff = max(fE_ff, fE_max_stress(s, mat))
        fE_iff_2D = max(fE_iff, fE_hashin_2D(s, mat)[1])
        fE_iff = max(fE_iff, fE_hashin(s_whole, mat)[1])

        stresses.append(s)
        local_strains.append(strain_local)
        Qs.append(Q)
        global_strains.append(strains_global)
        global_stresses.append(s_global)
        
        
    results["stresses"] = stresses
    results["fE_ff"] = fE_ff
    results["strains_local"] = local_strains
    results["fE_iff"] = fE_iff
    results["fE_iff_2D"] = fE_iff
    
    results["A"] = A
    results["Q"] = Qs
    results["global_strains"] = global_strains
    results["global_stresses"] =  global_stresses
    results["loads"] = loads
    
    return results

def loadcase(x):
    p = 0
    Fx = 0
    T = 0
    if x == 1:
        p = 8
    elif x == 2:
        Fx = 100E3
    elif x == 3:
        T = 2E7
    elif x == 4:
        p = 8
        Fx = 100E3
    elif x == 5:
        Fx = 100E3
        T = 2E7
    return p, Fx, T
            
        

In [2]:
# Case from website, glass fiber composite

m1=matlib.get('E-glass/Epoxy')
test_layup1 = [{'mat':m1, 'ori':-45, 'thi':1.0},
              {'mat':m1, 'ori':+45, 'thi':1.0}]
test_layup2 = [{'mat':m1, 'ori':0, 'thi':1.0},
              {'mat':m1, 'ori':0, 'thi':1.0}]

A = computeA(test_layup1)
R, P = 100, 2

Nx=P*R/2
Ny=P*R
Nxy=0
loads=[Nx,Ny,Nxy]

strains_global=np.linalg.solve(A,loads)

# Finding stresses in layer one of the laminate
layer1 = test_layup1[0]
angle = layer1["ori"]

strains_local = np.matmul(T2De(angle), strains_global)

Q = Q2D(m1)
Q_global = Q2Dtransform(Q, angle)
stresses_local = np.matmul(Q, strains_local)
stresses_global = np.matmul(Q_global, strains_global)
print("A", A)
print("")
print("Q", Q)
print("")
print("global strains", strains_global)

print("123 strains", strains_local)
#print("")
#print("local stresses", stresses_local)
#print("global stresses", stresses_global)


s1 = stresses_local[0]

NameError: name 'matlib' is not defined

In [ ]:
# Case for website with simplyfied method, glass fiber composite
R, P, Fx, T = 100, 2, 0, 0

m1=matlib.get('E-glass/Epoxy')

layup=[{'mat':m1, 'ori':-45, 'thi':1.0},
        {'mat':m1, 'ori':+45, 'thi':1.0}]


results = solve(layup,R=R,P=P,Fx=Fx,T=T)

print("A", results["A"])
print("")
print("Q", results["Q"][0])
print("")
print("Global strains", results["global_strains"][0])
print("")
print("123 strains:", results["strains_local"][0])
print("")
print("global stresses", results["global_stresses"][0])


#print("stresses:", stresses)
#print("max stress:", fE_ff)
#print("hashin iff", fE_iff)


In [8]:
# Thickness optimization

P,Fx,T = loadcase(1)
R = 75

theta1 = 30
theta2 = 80

x_er = []
y_er = []

set_plot_props()

forhold = np.linspace(1.00,5, 1000)
fE_iff_er = []
min_forhold = 0
min_fE = 100
t = 1

for f in forhold:
    t1 = t - t/f
    t2 = t/f
    #print(t1 + t2)

    layup     = [{'mat':m,  'ori':  theta1,  'thi':t1/2},
                 {'mat':m,  'ori': -theta1,  'thi':t1/2},
                 {'mat':m,  'ori':  theta2,  'thi':t2/2},
                 {'mat':m,  'ori': -theta2,  'thi':t2/2}]

    layup2     = [{'mat':m2,  'ori':  theta1,  'thi':t1/2},
                 {'mat':m2,  'ori': -theta1,  'thi':t1/2},
                 {'mat':m2,  'ori':  theta2,  'thi':t2/2},
                 {'mat':m2,  'ori': -theta2,  'thi':t2/2}]

    res = solve(layup,R, P=P*1.2, Fx=Fx*1.2, T=T*1.2)
    res_ff = solve(layup,R, P=P*3, Fx=Fx*3, T=T*3)

    fE = res["fE_iff"]
    #fE_ff = res_ff["fE_ff"]

    if fE < min_fE:
        min_fE = fE
        min_forhold = t1/t2
        #print(fE)


    fE_iff_er.append(fE)

y_er.append(fE_iff_er)
#print(f"{fE_iff=}")
#plt.plot(forhold, fE_iff_er)
for y,l in zip(y_er, ["30-50", "30-60", "30-70", "30-80"]:
    
    plt.plot(forhold, y, label=l)
    #print(min(fE_iff_er))
    #print("min forhold", min_forhold)
    #print("min forhold", min_pos)


NameError: name 'fE2DMS' is not defined